# **Computation of the optimal value of makespan and regularity**

## DOcplex Python API installation

First, we install the docplex python API which we will use for modelling our problem and solving it. 

In [1]:
import sys
try:
    import docplex.mp
except:
    if hasattr(sys, 'real_prefix'):
        !pip install docplex -q
        print("!pip install docplex -q...")
    else:
        !pip install --user docplex -q
        print("!pip install --user docplex -q...")
        
print("End of the intallation of python API...")

End of the intallation of python API...


In [2]:
#############################
### Import libraries ###
#############################

from docplex.cp.model import *
from docplex.cp.config import get_default
import numpy as np
from Solver import *
from User import *
import time

In [3]:
#############################
### Essentials functions ###
#############################
import FunctionMain as fm

In [4]:
# ----------------- Parameters

# The file to consider
file = './file_with_optimal_val/la02.txt'

# The number of solutions that we will have in the solver during the first iteration
k = 10

# The number of layer which is fixed
nb_layers = 5

# Variable which display the solution
display_sol = True

# Variable which display the start (in a vector)
display_start = False

# Variable which display the start (in a matrix)
display_matrix = False

# Time stop criterion 
tps_max = 1000

# Number of iteration stop criterion
it_max = 10

# Initialization of the number of iteration
it = 1

# The number of solutions that we will have in the solver after the first iteration
k_k = 10

list_search_type = ["DepthFirst", "Restart", "MultiPoint", "IterativeDiving"]

In [6]:
#############################
### Main program ###
#############################

print("\n--------Main program is loading...---------")

# --------- Interaction with the solver
data = []
n, m, data, T_machine, T_duration, duration, optimalval = fm.get_data_from_file(file)

# --------- Call Solver constructor in Solver.py and create the tasks of the model
model, solver , tasks = fm.initialize_solver(data, n, m, duration)

# ------------ Solve the model
print("\nSolving the model...")

# list_tasks = solver.get_tasks()
# tasks = list_variables[0]

# ------------ Add constraints to the solver

print("\nAdding precedence constraints to the solver...")
# Add precedence constraints
for i in range(n):
    for j in range(1,m):
        solver.add_constraint(model, end_before_start(tasks[i][T_machine[i*m + j-1]], tasks[i][T_machine[i*m + j]]))
print("Precedence constraints added !")

print("\nAdding disjunctive constraints to the solver...")
# Add disjunctive constraints 
for machine in range(m):
    solver.add_constraint(model, no_overlap([tasks[i][machine] for i in range(n)]))
print("Disjunctive constraints added !")

print("\nAdding objective function to the solver...")

makespan = max([model.end_of(tasks[i][T_machine[i*m + m-1]]) for i in range(n)])
# print("Makespan = ", makespan)

# Add objective function
waiting_time = [[] for i in range(m)]
machines = [[] for i in range(m)]
machinesTemp = [[] for i in range(m)]

SortedSof = [[None for i in range(n)] for j in range(m)]
SortedEof = [[None for i in range(n)] for j in range(m)]

P = [[None for i in range(n)] for j in range(m)]

list_obj = []

for j in range(m):
    for i in range(n):

        machines[j].append(tasks[i][j])
        
        SortedSof[j][i] = model.integer_var(min=0, max=2*optimalval)
        SortedEof[j][i] = model.integer_var(min=0, max=2*optimalval)
    

for j in range(m):
    for i in range(n-1):

        solver.add_constraint(model, SortedSof[j][i] < SortedSof[j][i+1])
        solver.add_constraint(model, SortedEof[j][i] < SortedEof[j][i+1])

        MachinesStart = [SortedSof[j][i] == model.start_of(machines[j][k]) for k in range(n)]
        solver.add_constraint(model, logical_or(MachinesStart))

        MachinesEnd = [SortedEof[j][i] == model.end_of(machines[j][k]) for k in range(n)]
        solver.add_constraint(model, logical_or(MachinesEnd))

        waiting_time[j].append(SortedSof[j][i+1] - SortedEof[j][i])


# for machine in range(m):
#     sum = 0
#     for i in range(len(waiting_time)-1):
#         for j in range(i+1, len(waiting_time[i])):
#             sum += abs(waiting_time[machine][j] - waiting_time[machine][i])
#     list_obj.append(sum)

# sum = 0
# for i in range(len(list_obj)):
#     sum += list_obj[i]

solver.add_constraint(model, model.minimize(2 * makespan))


# Solve the model.
msol = model.solve(TimeLimit=5)
# msol = model.solve(TimeLimit=5, LogVerbosity="Quiet")
print(type(msol)) #CpoSolveResult
print("objective value : ", msol.get_objective_value())
print("all_var_solutions : ",msol.get_all_var_solutions())
print("solution : ", msol.get_solution())
# j=0
# for i in msol:
#     j += 1
#     i.write()
# # print(j)


# ------------ Display the result
# fm.display_solution(msol, display_sol)
print("Model solved !")

# # ---------------- Interaction with the user
# print("\n--------Interaction with the user...---------")
# print("\nCreating the user...")
# user = User()
# print("User created !")

# #Get the tasks of the model
# tasks = solver.get_tasks()

# list_indice, list_obj, pref, list_layers, list_equal = fm.user_preferences(msol, user, nb_layers, n, m)

# # Vector of the start time of each task of each preference
# starts = user.start_pref(n, m, tasks, display_start)

# # Matrix of the start time of each task of each preference
# matrix = user.matrix_pref(n, m, display_matrix)

# # Testing the order of preferences and the differences between solutions
# fm.test(n, m, user)

# print("list layers : ",list_layers)



--------Main program is loading...---------

Creating the model variables...
Model tasks_by_jobs created !

Solver initialized !

Solving the model...

Adding precedence constraints to the solver...
Precedence constraints added !

Adding disjunctive constraints to the solver...
Disjunctive constraints added !

Adding objective function to the solver...
 ! --------------------------------------------------- CP Optimizer 22.1.1.0 --
 ! Minimization problem - 155 variables, 225 constraints
 ! TimeLimit            = 5
 ! Initial process time : 0.02s (0.02s extraction + 0.00s propagation)
 !  . Log search space  : 1316.8 (before), 1316.8 (after)
 !  . Memory usage      : 1.0 MB (before), 1.0 MB (after)
 ! Using parallel search with 8 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0        155                 -
 + New bound is 788
 ! Using iterative diving.
 ! Us

                    16000        100    5       361 != _INT_73
                    17000        100    5       526 != _INT_49
                    18000         90    5       103 != _INT_29
                    19000         89    5       502 != _INT_91
                    20000         92    5         6 != _INT_88
                    21000         89    5        67 != _INT_68
                     1001        100    6   F   532  = startOf(T2-3)
                     2000         92    6   F   805  = _INT_97
                     3000         89    6   F   768  = _INT_18
                     4000         92    6   F   360  = _INT_58
                     5000         87    6        82 != _INT_90
                     6000         85    6   F   176  = _INT_41
                     7000         91    6   F   236  = _INT_14
                     8000         88    6       307 != _INT_62
                     9000         91    6        63  = startOf(T5-1)
                    10000         97    6  

                    32000         98    3   F    29  = _INT_61
                    33000         98    3   F    16  = _INT_68
                    34000        100    3   F    19  = _INT_44
                    35000        100    3   F    48  = _INT_72
                    36000         97    3       538 != _INT_46
                    37000         90    3       431  = startOf(T9-3)
                    38000         96    3   F   457  = _INT_95
                    39000         89    3         1 != _INT_63
                    40000         93    3   F   110  = _INT_30
                    41000         88    3        27 != _INT_73
                    42000         89    3   F   708  = _INT_46
                    43000         89    3   F    46  = _INT_56
                    44000         99    3       125 != _INT_24
                    45000         99    3        28 != _INT_44
                    46000         98    3   F     1  = _INT_61
                    47000        100    3   F   9

                    57000         77    6   F    98  = _INT_83
                    58000         77    6       388 != _INT_8
                    59000         75    6       206 != _INT_91
                    60000         78    6        16 != _INT_98
                    61000         97    6   F   948  = _INT_89
                    62000         95    6   F   432  = _INT_98
                    63000         95    6   F     2  = _INT_23
                    64000         89    6      1158 != _INT_75
                    65000         86    6   F    16  = _INT_69
                    66000         98    6   F    33  = _INT_23
                    67000         98    6   F    56  = _INT_9
                    68000         97    6   F   232  = _INT_69
                    69000         90    6       509 != _INT_52
                    70000         93    6   F   251  = _INT_37
                    71000         92    6       932 != _INT_32
                    72000         89    6       874 != _I

 ! Time = 3.53s, Memory usage = 9.8 MB
 ! Current bound is 788
 !          Best Branches  Non-fixed    W       Branch decision
                    72000         73    3       290 != _INT_8
                    73000         70    3       138 != _INT_8
                    74000         67    3       483 != _INT_89
                    75000         67    3       630  = startOf(T4-1)
                    76000         97    3   F    12  = _INT_16
                    77000        100    3   F    13  = _INT_71
                    78000         91    3       547  = startOf(T6-0)
                    79000         89    3   F   493  = _INT_38
                    80000         96    3       942 != _INT_74
                    81000         95    3   F   700  = _INT_54
                    82000         94    3       202 != _INT_7
                    83000        100    3        28 != _INT_63
                    84000        100    3        72 != _INT_54
                    85000         99    3   F

                    77000         83    6   F  1084  = _INT_5
                    78000         82    6       568 != _INT_31
 ! Time = 3.65s, Memory usage = 9.7 MB
 ! Current bound is 788
 !          Best Branches  Non-fixed    W       Branch decision
                    79000         82    6        38 != _INT_44
                    80000         81    6   F   312  = _INT_64
                    81000         79    6   F   306  = _INT_52
                    82000         78    6       932 != _INT_4
                    83000         90    6   F  1147  = _INT_89
                    84000         88    6        86 != _INT_55
                    85000         85    6   F   975  = _INT_36
                    86000         85    6   F   429  = _INT_25
                    87000         82    6   F   757  = _INT_36
                    88000        100    6   F   251  = _INT_11
                    89000         99    6       203  = startOf(T2-2)
                    90000        100    6      120

                    96000         90    3   F   324  = _INT_16
                    97000         79    3        45 != _INT_50
                    98000         79    3   F    48  = _INT_50
                    99000         84    3         8 != _INT_50
 ! Time = 4.88s, Memory usage = 9.8 MB
 ! Current bound is 788
 !          Best Branches  Non-fixed    W       Branch decision
                     100k         94    3        73 != _INT_44
                     101k         94    3   F   332  = _INT_71
                     102k         88    3       497 != _INT_81
                     103k         88    3      1180 != _INT_66
                     104k         94    3   F   846  = _INT_17
                     105k         93    3       933 != _INT_74
                     106k         92    3   F   109  = _INT_87
                     107k         92    3      1027 != _INT_50
                     108k         90    3        49 != _INT_7
                     109k         98    3   F    76  = 

<class 'docplex.cp.solution.CpoSolveResult'>
objective value :  None
all_var_solutions :  None
solution :  None
Model solved !
